In [1]:
import os
import math
import shutil
import numpy as np
from multiprocessing import Pool
from keras.optimizers import RMSprop
from keras.losses import categorical_crossentropy
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
GPU = '0'
RND = 0
RUN = 'D3'
OUT_DIR = 'out_1m/'
TRAIN_TMP_DIR = OUT_DIR + '/train'
INPUT_DIR = '/d2/caches/tf-speech/train/audio'
TENSORBOARD_DIR = '/tensorboard/tf-speech/%s' % RUN
MODELS_DIR = '%s/models/%s' % (OUT_DIR, RUN)
INPUT_SIZE = (64, 64, 1)  # n_mels x width x 1ch

N_VAL_SAMPLES = 3000
N_TRAIN_SAMPLES = 1000000  # how many training samples to generate

In [3]:
# make only specific GPU to be utilized
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = GPU

In [4]:
%run 'lib.ipynb'
%run 'data-generator.ipynb'
%run 'models.ipynb'

In [5]:
# remove tensorboard data
if os.path.isdir(TENSORBOARD_DIR): shutil.rmtree(TENSORBOARD_DIR)

In [6]:
# load val data

val_X_path = OUT_DIR + '/val_X.npy'
val_Y_path = OUT_DIR + '/val_Y.npy'

val_X = np.load(val_X_path)
val_Y = np.load(val_Y_path)

assert len(val_X) == len(val_Y)
print('val samples: %d' % len(val_X))

val samples: 3000


In [7]:
# load training data

train_X_file = '%s/train_X.mem' % OUT_DIR
train_Y_file = '%s/train_Y.mem' % OUT_DIR

train_X = np.memmap(
    train_X_file, np.float32, 'r', shape=(N_TRAIN_SAMPLES, ) + INPUT_SIZE)
train_Y = np.memmap(
    train_Y_file, np.float32, 'r', shape=(N_TRAIN_SAMPLES, len(LABELS)))

assert len(train_X) == len(train_Y)
print('training samples: %d' % len(train_X))

training samples: 1000000


In [8]:
# create model
model = Model_2(input_size=INPUT_SIZE, output_size=len(LABELS))
model.build()
optimizer = RMSprop(lr=1e-3)
model.m.compile(
    optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy']\
)

In [9]:
# create models dir
if os.path.isdir(MODELS_DIR): shutil.rmtree(MODELS_DIR)
os.makedirs(MODELS_DIR)

In [10]:
# train model

N_PER_BATCH = 500
STEPS_PER_EPOCH = len(
    train_X) // N_PER_BATCH // 10  # last number splits train set into # epochs
N_EPOCHS = 100

print('# samples per epoch: %d\n' % (STEPS_PER_EPOCH * N_PER_BATCH))


def train_generator(n_per_batch):
    start_i = 0
    while True:
        if start_i >= len(train_X): start_i = 0
        batch_X = train_X[start_i:start_i + n_per_batch]
        batch_Y = train_Y[start_i:start_i + n_per_batch]
        yield (batch_X, batch_Y)
        start_i += n_per_batch


model.m.fit_generator(
    train_generator(N_PER_BATCH),
    STEPS_PER_EPOCH,
    epochs=N_EPOCHS,
    validation_data=(val_X, val_Y),
    callbacks=[
        TensorBoard(log_dir=TENSORBOARD_DIR),
        ModelCheckpoint(
            MODELS_DIR +
            '/e{epoch:03d}-l={loss:.5f}-vl={val_loss:.5f}-a={acc:.5f}-va={val_acc:.5f}.h5',
            monitor='val_acc',
            verbose=0,
            save_best_only=False,
            save_weights_only=False,
            mode='auto'),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=2,
            min_lr=1e-8,
            verbose=1),
        EarlyStopping(
            monitor='val_acc',
            min_delta=0.00001,
            patience=5,
            verbose=1,
            mode='auto')
    ])

# samples per epoch: 100000

Epoch 1/100
200/200 [==============================] - 15s - loss: 2.1088 - acc: 0.2789 - val_loss: 1.0974 - val_acc: 0.6630
Epoch 2/100
200/200 [==============================] - 14s - loss: 1.4686 - acc: 0.5098 - val_loss: 0.7354 - val_acc: 0.7613
Epoch 3/100
200/200 [==============================] - 15s - loss: 1.2078 - acc: 0.6032 - val_loss: 0.5760 - val_acc: 0.8117
Epoch 4/100
200/200 [==============================] - 15s - loss: 1.0490 - acc: 0.6581 - val_loss: 0.5012 - val_acc: 0.8330
Epoch 5/100
200/200 [==============================] - 15s - loss: 0.8959 - acc: 0.7088 - val_loss: 0.4552 - val_acc: 0.8523
Epoch 6/100
200/200 [==============================] - 15s - loss: 0.8698 - acc: 0.7188 - val_loss: 0.3924 - val_acc: 0.8780
Epoch 7/100
200/200 [==============================] - 15s - loss: 0.7446 - acc: 0.7587 - val_loss: 0.3647 - val_acc: 0.8857
Epoch 8/100
200/200 [==============================] - 15s - loss: 0.6931 - acc: 0.7777 - val_lo